In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
import gc
import os
import matplotlib.pyplot as plt
import seaborn as sns 
import lightgbm as lgb
import itertools
import pickle, gzip
import glob
from sklearn.preprocessing import StandardScaler
from tsfresh.feature_extraction import extract_features
np.warnings.filterwarnings('ignore')
import dask.dataframe as dd
import missingno as msno
from pandasql import sqldf
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import KFold
import matplotlib.gridspec as gridspec
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

In [2]:
#Always seed the randomness of this universe
np.random.seed(51)

In [3]:
%%time
pred1 = dd.read_csv('test_pred_20.csv')
pred2 = dd.read_csv('test_pred_22.csv')
pred1 = pred1.compute()
pred2 = pred2.compute()
print(pred1.shape,pred2.shape)

(3492890, 16) (3492890, 16)
CPU times: user 32.5 s, sys: 2.88 s, total: 35.4 s
Wall time: 7.75 s


In [4]:
%%time
pred1 = pred1.reset_index(drop=True)
pred2 = pred2.reset_index(drop=True)

CPU times: user 145 ms, sys: 312 ms, total: 457 ms
Wall time: 455 ms


In [5]:
%%time
test_ensemble = pd.DataFrame()

CPU times: user 337 µs, sys: 0 ns, total: 337 µs
Wall time: 341 µs


In [6]:
coef_lgb = 0.8
coef_mlp = 0.2

In [7]:
%%time
test_ensemble = pred1.iloc[:,1:]*coef_lgb + pred2.iloc[:,1:]*coef_mlp

CPU times: user 334 ms, sys: 584 ms, total: 919 ms
Wall time: 916 ms


In [8]:
print(test_ensemble.shape)

(3492890, 15)


In [9]:
test_ensemble['object_id'] = pred1['object_id']

In [10]:
print(test_ensemble.shape)

(3492890, 16)


In [11]:
test_ensemble = test_ensemble[list(pred1.columns)]

In [12]:
test_ensemble.tail()

,object_id,class_6,class_15,class_16,class_42,class_52,class_53,class_62,class_64,class_65,class_67,class_88,class_90,class_92,class_95,class_99
3492885,130787966,0.000177,0.355040,0.000320,0.189941,0.228110,0.000343,0.081297,0.000124,0.002043,0.012705,0.003562,0.125831,0.000164,0.000344,0.245541
3492886,130787971,0.000252,0.014179,0.000268,0.061153,0.191013,0.000314,0.019944,0.359429,0.000555,0.031331,0.001038,0.316647,0.000254,0.003622,0.233665
3492887,130787974,0.000864,0.033511,0.000185,0.840147,0.016127,0.000492,0.090778,0.000056,0.000282,0.001402,0.000716,0.013580,0.000233,0.001628,0.110447
3492888,130788053,0.000068,0.750243,0.000092,0.125583,0.006922,0.000091,0.010652,0.073527,0.000297,0.000467,0.000126,0.031409,0.000075,0.000449,0.152445
3492889,130788054,0.000134,0.045608,0.000148,0.324511,0.416499,0.000261,0.038736,0.000135,0.000303,0.012286,0.000905,0.159904,0.000151,0.000419,0.226362


In [13]:
%%time
test_ensemble.to_csv('test_pred_26.csv',index=False)

CPU times: user 1min 33s, sys: 1.24 s, total: 1min 34s
Wall time: 1min 34s


In [14]:
!kaggle competitions submit -c PLAsTiCC-2018 -f test_pred_26.csv -m "Message"

100%|█████████████████████████████████████| 1.06G/1.06G [2:39:25<00:00, 119kB/s]
Successfully submitted to PLAsTiCC Astronomical Classification